En primer lugar preparamos el entorno importando las librerías que se van a utilizar

In [57]:
# Manejo de datos
import pandas as pd
import numpy as np

# Texto y similitud
from sklearn.metrics.pairwise import cosine_similarity

# Visualización
import matplotlib.pyplot as plt
import seaborn as sns

# Utilidades
import os
import pickle
from collections import Counter, defaultdict

# OpenAI embeddings
from openai import OpenAI
from dotenv import load_dotenv

import json
from numpy.linalg import norm
import time

En primer lugar realizamos la limpieza del csv resultante del formulario que se encuentra en la carpeta llamada "data"

In [89]:
# corrected relative path (ensure the 'data' folder is one level up from the notebook)
registros = pd.read_csv("..\data\Graph-Match _ CIS 2026-I0(Sheet1) (1).csv", sep=";", encoding="latin-1")

In [ ]:
registros.columns.all

In [90]:
#eliminamos las columnas que están vacías
registros = registros.dropna(axis=1, how="all")
#eliminamos las filas que no aceptaron participar en la actividad
registros = registros[registros["Â¿Aceptas participar en esta actividad?"] != "No"]
registros.head(5)

,ï»¿Id,Start time,Completion time,Email,Name,Â¿Aceptas participar en esta actividad?,Points - Â¿Aceptas participar en esta actividad?,nombre,Points - Â¿ CÃ³mo te llamas ?Â,contacto,...,Points - Â¿Con cuÃ¡l de estos personajes te identificas?1,pareja_fav,Points - CuÃ¡l de estas parejas es tu favorita?,descripcion_mi_tipo,Points - Describe tu tipo ideal !! (FÃ­sica y emocionalmente),motivacion,Points - Â¿QuÃ© te motivÃ³ a participar?,asistencia,Points - AsistirÃ­as a una charla/bootcamp/taller en el que aprenderÃ­as sobre todos los conceptos de inteligencia computacional detrÃ¡s de esta actividad?(Sistemas de recomendaciÃ³n hÃ­bridos/funciones de similitu,Points - Â¡Ãnete a nuestro grupo estudiantil!\n\nhttps://forms.office.com/r/bGSefEJgfs\n
1,2,11/02/2026 9:00,11/02/2026 9:03,jcpombo@uninorte.edu.co,JUAN CAMILO POMBO MUÃOZ,Si,0.0,Juan Camilo Pombo,0.0,3042147371,...,0.0,Option 7,0.0,Que pueda darme atenciÃ³n y sepa escuchar las ...,0.0,Apoyar el desarrollo de actividades con fines ...,0.0,Si,0.0,0.0
2,3,11/02/2026 8:59,11/02/2026 9:12,maldonadoal@uninorte.edu.co,Luis Alberto Maldonado Cano,Si,0.0,Luis Maldonado,0.0,3045533410,...,0.0,Option 2,0.0,Una mujer autÃ©ntica que no tenga miedo a demo...,0.0,DiversiÃ³n por San ValentÃ­n,0.0,Si,0.0,0.0
3,4,11/02/2026 9:17,11/02/2026 9:26,marimonmd@uninorte.edu.co,DAVID MANUEL MARIMON ESCOBAR,Si,0.0,David Marimon,0.0,3023800569,...,0.0,Option 8,0.0,"Una mujer tranquila y empÃ¡tica, de presencia ...",0.0,Apoyar el desarrollo de actividades con fines ...,0.0,Si,0.0,0.0
4,5,11/02/2026 9:27,11/02/2026 9:33,marianaserrato@uninorte.edu.co,MARIANA SERRATO MEJIA,Si,0.0,Mariana Serrato Mejia,0.0,@marser1610,...,0.0,Option 1,0.0,Una persona con madurez emocional y dispuesta ...,0.0,DiversiÃ³n por San ValentÃ­n,0.0,Si,0.0,0.0
5,6,11/02/2026 9:27,11/02/2026 9:34,orarroyo@uninorte.edu.co,ORLANDO RAFAEL ARROYO ESCALANTE,Si,0.0,Orlando,0.0,3042656793 o @orlandorarroyoe_,...,0.0,Option 7,0.0,Mi tipo de persona es una persona que le guste...,0.0,Curiosidad,0.0,Si,0.0,0.0


In [91]:
#Colapsamos las columnas de arquetipos en una sola 
registros["me_gusta"] = registros["me_gusta"].fillna(registros["me_gusta1"])
registros["me_identifico"] = registros["me_identifico"].fillna(registros["me_identifico1"])

In [92]:
#corrección:
registros["genero_interes"] = registros["genero_interes"].replace({
    'Mujeres': 'Mujer',
    'Hombres': 'Hombre'
})

In [ ]:
registros["genero_interes"].head(25)

A continuación convertiremos cada fila/registro en un nodo

In [93]:
usuarios = {}

for index, row in registros.iterrows():
    user = {
       "metadata":{
           "nombre": row["nombre"],
           "contacto": row["contacto"],
           "correo": row["Email"],
           "división": row["div"],
           "rango_edad": row["edad"],
           "género": row["genero"],
           "gen_interes": row["genero_interes"],
           "algo_serio": row["algo_serio"],
           "tipo_relacion": row["tipo"]
       },
       "var_estructurales": {
           "géneros_musica": row["musica"].split(";"),
           "planes_finde": row["finde"].split(";"),
           "tiempo_libre": row["freetime"].split(";"),
           "color": row["color"],
           
       },
       "escalas": {
           "nivel_extrovertido": row["extrover"],
           "nivel_emocional": row["emo"],
           "nivel_romantico": row["romantico"]
       },
       "love_and_roles": {
           "love_language_dar": row["love_language_dar"],
           "love_language_recibir": row["love_language_recibir"]
       },
       "arquetipo_visual": {
           "arquetipo_indentidad": row["me_identifico"],
           "arquetipo_ideal": row["me_gusta"],
           "pareja_favorita": row["pareja_fav"]
           
       },
       "descripcion": {
           "descripción_propia": row["mi_descripcion"],
           "descripción_tipo": row["descripcion_mi_tipo"]
       },
       "embeddings": {
           "propia": None,
           "tipo": None
       },
       "feedback": {
           "asistirá": row["asistencia"],
           "motivación": row["motivacion"]
       }
   }
   
    # usamos el índice como ID único
    usuarios[f"id_{index}"] = user

In [65]:
from sentence_transformers import SentenceTransformer
from sklearn.metrics.pairwise import cosine_similarity# Cargamos el modelo (se descarga una sola vez, pesa unos 400MB)
model_vibe = SentenceTransformer('paraphrase-multilingual-MiniLM-L12-v2')

Loading weights:   0%|          | 0/199 [00:00<?, ?it/s]

BertModel LOAD REPORT from: sentence-transformers/paraphrase-multilingual-MiniLM-L12-v2
Key                     | Status     |  | 
------------------------+------------+--+-
embeddings.position_ids | UNEXPECTED |  | 

Notes:
- UNEXPECTED	:can be ignored when loading from different task/architecture; not ok if you expect identical arch.


In [94]:
#Generamos los embeddings localmente:
for uid, user in usuarios.items():
    # Guardamos el vector directamente en el diccionario del usuario
    usuarios[uid]["embeddings"]["propia"] = model_vibe.encode(user["descripcion"]["descripción_propia"])
    usuarios[uid]["embeddings"]["tipo"] = model_vibe.encode(user["descripcion"]["descripción_tipo"])

In [ ]:
usuarios["id_100"]["embeddings"]["propia"]

A continuación haremos un ciclo for, para realizar las comparaciones pertinentes, es importante tener en cuenta no realizar una comparación en bruto dado que es exactamente igual para este caso comparar a la persona A con la persona B que la persona B con la persona A; por lo tanto usaremos itertools, un módulo de python especializado en funciones que devuelven objetos iterables diseñado para una ejecución rápida haciendo uso eficiente de la memoria.

In [96]:
from itertools import combinations


In [ ]:
personas = {
    "ID1": {"nombre": "Ana", "gustos": ["Cine", "Bici","coco","ron","vino","playa","azul"]},
    "ID2": {"nombre": "Beto", "gustos": ["Bici", "Libros","vino","agua","rio","azul"]},
    "ID3": {"nombre": "Carla", "gustos": ["Cine", "Libros"]}
}

In [ ]:
#ejemplo intersección:
# Intersección gustos (música) y hobbies (planes/tiempo libre)
gustos_comun = len(set(personas["ID1"]["gustos"]) & set(personas["ID2"]["gustos"]))
print(f"Gustos en común entre {personas['ID1']['nombre']} y {personas['ID2']['nombre']}: {gustos_comun}")

Gustos en común entre Ana y Beto: 3


In [97]:
# 1. FilTROS DE EXCLUSIÓN
def validar_interes_mutuo(u1, u2):
     # a) género de interés vs género propio: Si a u1 le interesan hombres y u2 es mujer, saltamos, del mismo modo, si a u2 le interesan mujeres 
     # # y u1 es hombre, también saltamos.
     interes_1_por_2 = (u1["metadata"]["gen_interes"] == u2["metadata"]["género"]) or (u1["metadata"]["gen_interes"] == "Me da igual")
     interes_2_por_1 = (u2["metadata"]["gen_interes"] == u1["metadata"]["género"]) or (u2["metadata"]["gen_interes"] == "Me da igual")   
     # Si uno de los dos no está interesado en el género del otro, abortamos el match
     if not (interes_1_por_2 and interes_2_por_1):
         print(f"SE ABORTA: {u1['metadata']['nombre']} + {u2['metadata']['nombre']} porg.")
         return False
     elif u1["metadata"]["algo_serio"] != u2["metadata"]["algo_serio"]:
         # Borramos el match??? 'continue' o seguimos con menos puntos?????*************
         print(f"SE ABORTA: {u1['metadata']['nombre']} + {u2['metadata']['nombre']} por algo serio.")
         return False
     elif u1["metadata"]["rango_edad"] != u2["metadata"]["rango_edad"]:
        print(f"SE ABORTA: {u1['metadata']['nombre']} + {u2['metadata']['nombre']} por edad.")
        return False
     else:
        return True

In [109]:
print(usuarios["id_38"]["metadata"]["tipo_relacion"])
print(usuarios["id_20"]["metadata"]["tipo_relacion"])

Los iguales se entienden
Los opuestos se atraen


In [106]:
print(usuarios["id_38"]["descripcion"]["descripción_tipo"])
print("*****************************************************")
print(usuarios["id_64"]["descripcion"]["descripción_propia"])
print("*****************************************************")

print(usuarios["id_64"]["descripcion"]["descripción_tipo"])
print("*****************************************************")

print(usuarios["id_38"]["descripcion"]["descripción_propia"])

Alto, un poco friki, extrovertido, participativo, mayor que yo, carismÃ¡tico, que respete a las mujeres, que no sea coqueto, que sea atento, contextura entre delgada y media, inteligente y estudioso.
*****************************************************
Delgado, cabello oscuro, ojos cafÃ© y tez clara, tranquilo, reservado, mÃ¡s de escuchar que de hablar, valoro la honestidad y la buena energÃ­a.
Estilo sencillo, sin exagerar, pero bien cuidado. No lo cuento todo de entrada prefiero que lo descubran.
*****************************************************
me llama la atenciÃ³n el cabello lacio u ondulado, pero lo que realmente me atrae es la actitud. Me gusta alguien inteligente, con quien hablar sea sencillo y natural. Real, centrada y con ese algo que la destaque.
*****************************************************
Soy una persona tranquila y atenta que le gusta estar en casa y jugar videojuegos, estudiosa, con grandes aspiraciones, callada (pero me gusta hablar bastante) y a veces me

In [77]:
def calcular_puntos_love_language(u1, u2):
    def afinidad_direccional(da, recibe):
        set_da = set(da)
        set_recibe = set(recibe)
        interseccion = len(set_da & set_recibe)
        union = len(set_da | set_recibe)
        return (interseccion / union) if union > 0 else 0

    # ¿Qué tanto le da U1 a U2?
    afinidad_1_a_2 = afinidad_direccional(
        u1["love_and_roles"]["love_language_dar"], 
        u2["love_and_roles"]["love_language_recibir"]
    )
    
    # ¿Qué tanto le da U2 a U1?
    afinidad_2_a_1 = afinidad_direccional(
        u2["love_and_roles"]["love_language_dar"], 
        u1["love_and_roles"]["love_language_recibir"]
    )
    
    # Promediamos ambas direcciones y escalamos a 13 puntos
    return ((afinidad_1_a_2 + afinidad_2_a_1) / 2) * 13

Problema de proporcionalidad:
No es lo mismo coincidir en 2 de 2 (100% de afinidad) que en 2 de 10 (20% de afinidad).

Para solucionar esto, usaremos el Índice de Jaccard, que mide qué tan parecidas son dos listas teniendo en cuenta el tamaño de ambas.

In [78]:
def jaccard(lista1, lista2):
        s1, s2 = set(lista1), set(lista2)
        return len(s1 & s2) / len(s1 | s2) if (s1 | s2) else 0

In [79]:
def bloque_estructurado(u1, u2):
    # --- BLOQUE ESTRUCTURADO (45%) ---
    # Puntos max del bloque: 13+7+10+15 = 45 pts
    
    # Intersección gustos (música) y hobbies (planes/tiempo libre)
    pts_musica = jaccard(u1["var_estructurales"]["géneros_musica"], u2["var_estructurales"]["géneros_musica"]) * 5
    pts_hobbies = jaccard(u1["var_estructurales"]["planes_finde"], u2["var_estructurales"]["planes_finde"]) * 8
    pts_freetime = jaccard(u1["var_estructurales"]["tiempo_libre"], u2["var_estructurales"]["tiempo_libre"]) * 7
    
    
    # Love Languages (13 pts)
    pts_ll = calcular_puntos_love_language(u1, u2)
    
        
    # Escalas (12 pts) - Diferencia normalizada
    diff_promedio = sum([
        abs(u1["escalas"]["nivel_extrovertido"] - u2["escalas"]["nivel_extrovertido"]),
        abs(u1["escalas"]["nivel_emocional"] - u2["escalas"]["nivel_emocional"]),
        abs(u1["escalas"]["nivel_romantico"] - u2["escalas"]["nivel_romantico"])
    ]) / 3

    pref_u1 = u1["metadata"]["tipo_relacion"]
    pref_u2 = u2["metadata"]["tipo_relacion"]

    if pref_u1 == pref_u2 == "Los iguales se entienden":
        pts_escalas = (1 - (diff_promedio / 4)) * 12
    elif pref_u1 == pref_u2 == "Los opuestos se atraen":
        pts_escalas = (diff_promedio / 4) * 12
    else:
        # Si uno quiere igual y el otro opuesto, calculamos el punto medio de satisfacción
        # No los castigamos a cero, pero no llegan al máximo.
        pts_escalas = 6

    return pts_musica + pts_hobbies + pts_freetime + pts_ll + pts_escalas

In [80]:
def bloque_arquetipos(u1, u2):
    # --- BLOQUE ARQUETIPOS (20%) ---
    # Puntos max del bloque: 10+10+5 = 20 pts
    total = 0
    #verificamos si alguno de los 2 está nulo, porque eso significa que cuando seleccionaron 
    #género de interes marcaron "me da igual" y no seleccionaron arquetipo ideal.
    if pd.isna(u1["arquetipo_visual"]["arquetipo_ideal"]) and pd.isna(u2["arquetipo_visual"]["arquetipo_ideal"]):
        total+=20
    elif pd.isna(u2["arquetipo_visual"]["arquetipo_ideal"]):
        total+=5
    elif pd.isna(u1["arquetipo_visual"]["arquetipo_ideal"]):
        total+=5
    if u1["arquetipo_visual"]["arquetipo_ideal"] == u2["arquetipo_visual"]["arquetipo_indentidad"]: total += 10
    if u2["arquetipo_visual"]["arquetipo_ideal"] == u1["arquetipo_visual"]["arquetipo_indentidad"]: total += 10
    if u1["arquetipo_visual"]["pareja_favorita"] == u2["arquetipo_visual"]["pareja_favorita"]: total += 5
    return total

In [55]:
if True and True:
    print("hola")
elif True and True:
    print("holi")

hola


In [ ]:
#ESTE SUPER NOOOO BYEEE.
def bloque_embeddings(u1, u2):
    """
    Calcula compatibilidad por texto con normalización de rango real.
    Basado en el comportamiento empírico de OpenAI (rango 0.7 - 0.9).
    """
    # 1. Similitud cruzada (Lo que yo soy vs lo que tú buscas)
    sim_a_b = cosine_similarity(u1["embeddings"]["tipo"], u2["embeddings"]["propia"])
    sim_b_a = cosine_similarity(u2["embeddings"]["tipo"], u1["embeddings"]["propia"])
    
    similitud_promedio = (sim_a_b + sim_b_a) / 2

    # 2. Normalización de escala (Stretching)
    # Definimos el rango:
    min_real = 0.72  # Por debajo de esto, no hay afinidad real
    max_real = 0.92  # Por encima de esto, es un match excepcional
    
    if similitud_promedio <= min_real:
        pts = 0.0
    elif similitud_promedio >= max_real:
        pts = 30.0
    else:
        # Re-mapeo lineal: rango de [0.72, 0.92] a [0, 30]
        pts = (similitud_promedio - min_real) / (max_real - min_real) * 30
        
    return round(pts, 2)

In [81]:
def bloque_embeddings_local(u1, u2):
    # 1. Extraemos los vectores que ya deberían estar pre-calculados
    emb_propia_1 = u1["embeddings"]["propia"]
    emb_tipo_2 = u2["embeddings"]["tipo"]
    
    emb_propia_2 = u2["embeddings"]["propia"]
    emb_tipo_1 = u1["embeddings"]["tipo"]

    # 2. Similitud Coseno Cruzada
    sim1 = cosine_similarity([emb_propia_1], [emb_tipo_2])[0][0]
    sim2 = cosine_similarity([emb_propia_2], [emb_tipo_1])[0][0]
    
    sim_promedio = (sim1 + sim2) / 2

    umbral_min = 0.25  
    umbral_max = 0.58
    
    if sim_promedio <= umbral_min:
        return 0.0
    if sim_promedio >= umbral_max:
        return 30.0
        
    # Interpolación lineal para sacar el puntaje
    pts = (sim_promedio - umbral_min) / (umbral_max - umbral_min) * 30
    return pts

In [ ]:
users_ids = list(usuarios.keys())
lista_para_excel = []
murieron = 0
# 1. Contenedor para los rankings personalizados (ID -> Lista de matches)
rankings_individuales = {uid: [] for uid in usuarios.keys()}
for id_a, id_b in combinations(users_ids, 2):
    print(id_a, "vs", id_b)
    u1 = usuarios[id_a]
    u2 = usuarios[id_b]
    if not validar_interes_mutuo(u1, u2):
        murieron+=1
    else:
        # --- CÁLCULO DE SCORE ---
        puntos_estructurado = bloque_estructurado(u1, u2)       
        puntos_arquetipos = bloque_arquetipos(u1, u2)   
        puntos_texto = bloque_embeddings_local(u1, u2)
        score_final = puntos_estructurado + puntos_arquetipos + puntos_texto
        
        if score_final > 0:
            lista_para_excel.append({
                "Persona A": u1["metadata"]["nombre"],
                "ID A": id_a,
                "Persona B": u2["metadata"]["nombre"],
                "ID B": id_b,
                "Compatibilidad Total (%)": score_final, 
                "Puntos Estructurado (45)": puntos_estructurado,
                "Puntos Visuales (25)": puntos_arquetipos,
                "Puntos Texto (30)": puntos_texto,
                "Contacto A": u1["metadata"]["contacto"],
                "Contacto B": u2["metadata"]["contacto"]
            })
            # 2. ALMACENAMIENTO EN RANKING (Bidireccional)
        # Guardamos el match para la Persona A
        rankings_individuales[id_a].append({
            "match_con": u2["metadata"]["nombre"],
            "id_pareja": id_b,
            "compatibilidad": round(score_final, 4), # Guardamos 4 decimales para el orden interno
            "contacto": u2["metadata"]["contacto"],
            "desglose": {"est": puntos_estructurado, "vis": puntos_arquetipos, "txt": puntos_texto},
            "correo_destino": u1["metadata"]["correo"]
        })
        
        # Guardamos el match para la Persona B
        rankings_individuales[id_b].append({
            "match_con": u1["metadata"]["nombre"],
            "id_pareja": id_a,
            "compatibilidad": round(score_final, 4),
            "contacto": u1["metadata"]["contacto"],
            "desglose": {"est": puntos_estructurado, "vis": puntos_arquetipos, "txt": puntos_texto},
            "correo_destino": u2["metadata"]["correo"]
            
        })

# 3. ORDENAMIENTO DE PRECISIÓN
# Aquí es donde los decimales brillan: ordenamos de mayor a menor
for uid in rankings_individuales:
    rankings_individuales[uid] = sorted(
        rankings_individuales[uid], 
        key=lambda x: x["compatibilidad"], 
        reverse=True
    )

# Crear DataFrame y exportar
df_matches_viables = pd.DataFrame(lista_para_excel)

# Ordenamos el Excel general de mayor a menor compatibilidad global
df_matches_viables = df_matches_viables.sort_values(by="Compatibilidad Total (%)", ascending=False)

# Guardar a archivo
df_matches_viables.to_excel("Final_maches.xlsx", index=False)

In [ ]:
def calcular_puntos_vibe(t1, t2):
    emb1 = model_vibe.encode([t1])
    emb2 = model_vibe.encode([t2])
    sim = cosine_similarity(emb1, emb2)[0][0]
    
    # Ajustamos el umbral para este modelo específico
    # En este modelo, 0.35 es casi nada en común y 0.85 es casi lo mismo
    min_real, max_real = 0.25, 0.58
    
    if sim <= min_real:
        pts = 0.0
    elif sim >= max_real:
        pts = 30.0
    else:
        pts = (sim - min_real) / (max_real - min_real) * 30
    return sim, pts

# --- TEST DE CAMPO ---
pruebas = [
    ("Amo el gym, comer sano y madrugar.", "comer salchipapas a las 3am y dormir hasta tarde."),
    ("Me gusta leer libros de fantasía y quedarme en casa.", "Busco alguien rumbero que le guste el perreo y salir."),
    ("Soy fan de la tecnología y los videojuegos.", "Busco a alguien con quien ver pelis y jugar play.")
]

print("test de realismo...")
for p1, p2 in pruebas:
    s, pts = calcular_puntos_vibe(p1, p2)
    print(f"\nPersona A: {p1}\nPersona B: {p2}")
    print(f"Similitud: {s:.4f} | PUNTOS: {pts:.2f}/30")

SE CANCELA ESTA PARTE QUE ERA CON OPENAI porque NO HAY PLATA.

In [22]:
load_dotenv()  # Carga las variables de entorno desde el archivo .env
openai = OpenAI(api_key=os.getenv("OPEN_AI_API_KEY"))   


In [ ]:
def generar_embedding(texto, model="text-embedding-3-small"):
    """
    Recibe un texto y devuelve su embedding como lista de floats.
    """
    if texto is None or texto.strip() == "":
        return None

    response = openai.embeddings.create(
        model=model,
        input=texto
    )

    return response.data[0].embedding


In [20]:
def generar_embeddings_batch(textos, model="text-embedding-3-small"):
    """Genera embeddings para múltiples textos en batch"""
    textos_validos = []
    indices_validos = []
    
    for i, texto in enumerate(textos):
        if texto and texto.strip() != "":
            textos_validos.append(texto)
            indices_validos.append(i)
    
    if not textos_validos:
        return [None] * len(textos)
    
    try:
        response = openai.embeddings.create(
            model=model,
            input=textos_validos
        )
        
        embeddings = [None] * len(textos)
        for i, embedding_data in enumerate(response.data):
            idx_original = indices_validos[i]
            embeddings[idx_original] = embedding_data.embedding
        
        return embeddings
    
    except Exception as e:
        print(f"Error: {e}")
        return [None] * len(textos)

In [13]:
def guardar_embeddings(usuarios, filename="embeddings_cache.json"):
    """Guarda embeddings en archivo"""
    cache = {}
    for user_id, user in usuarios.items():
        cache[user_id] = {
            "propia": user["embeddings"]["propia"],
            "tipo": user["embeddings"]["tipo"]
        }
    
    with open(filename, 'w') as f:
        json.dump(cache, f)
    print(f"Embeddings guardados")

In [14]:
def cargar_embeddings(usuarios, filename="embeddings_cache.json"):
    """Carga embeddings desde archivo"""
    try:
        with open(filename, 'r') as f:
            cache = json.load(f)
        
        for user_id, embeddings in cache.items():
            if user_id in usuarios:
                usuarios[user_id]["embeddings"]["propia"] = embeddings["propia"]
                usuarios[user_id]["embeddings"]["tipo"] = embeddings["tipo"]
        
        print(f"Embeddings cargados")
        return True
    except FileNotFoundError:
        print(f"Generando embeddings nuevos...")
        return False

In [15]:
def cosine_similarity(embedding1, embedding2):
    """Calcula similitud coseno entre embeddings"""
    if embedding1 is None or embedding2 is None:
        return 0.0
    
    vec1 = np.array(embedding1)
    vec2 = np.array(embedding2)
    
    similarity = np.dot(vec1, vec2) / (norm(vec1) * norm(vec2))
    return float(similarity)

In [ ]:
if not cargar_embeddings(usuarios):
    # Si no existe, generar
    print("Generando embeddings...")
    
    all_textos_propios = [user["descripcion"]["descripción_propia"] 
                          for user in usuarios.values()]
    all_textos_tipos = [user["descripcion"]["descripción_tipo"] 
                       for user in usuarios.values()]
    
    embeddings_propios = generar_embeddings_batch(all_textos_propios)
    embeddings_tipos = generar_embeddings_batch(all_textos_tipos)
    
    for i, user_id in enumerate(usuarios.keys()):
        usuarios[user_id]["embeddings"]["propia"] = embeddings_propios[i]
        usuarios[user_id]["embeddings"]["tipo"] = embeddings_tipos[i]
    
    guardar_embeddings(usuarios)

PARTE DE AUTOMATIZACIÓN DE CORREOS:

In [110]:
def explicar_escalas(u1, u2):
    # Lógica de Escalas (Basado en lo que piensan de "Polos Opuestos")
    # Asumiendo que guardaste en el Excel si eran de "Iguales" o "Opuestos"
    pensa_1 = u1["metadata"]["tipo_relacion"] # "Iguales" u "Opuestos"
    pensa_2 = u2["metadata"]["tipo_relacion"]
    
    if pensa_1 == "Los opuestos se atraen" and pensa_2 == pensa_1:
        return "Ambos piensan que los opuestos se atraen, por eso el algoritmo buscó complementariedad en sus escalas de romanticismo, extroversión y qué tan emocionales son."
    elif pensa_1 == "Los iguales se entienden" and pensa_2 == pensa_1:
        return "Ambos piensan que los iguales se entienden mejor, por eso el algoritmo buscó una gran afinidad y similitud en sus personalidades y escalas."
    else:
        return "Tienen visiones distintas sobre el amor (uno cree en la similitud y otro en la complementariedad), ¡será interesante ver cómo equilibran sus personalidades!"

def explicar_arquetipos(yo, match):
    frases = []
    
# --- CASO 1: Tú eres su tipo ideal ---
    identidad_yo = yo["arquetipo_visual"]["arquetipo_indentidad"]
    ideal_match = pd.isna(match["arquetipo_visual"]["arquetipo_ideal"])
    
    if ideal_match == identidad_yo:
        frases.append(f"¡Eres su tipo ideal! El personaje con el que te identificaste ({identidad_yo}) es precisamente el que tu match prefiere.")
    elif ideal_match == "Me da igual":
        frases.append(f"A tu match le da igual el arquetipo físico, ¡valoró mucho más tu descripción y personalidad!")

    # --- CASO 2: El match es tu tipo ideal ---
    identidad_match = match["arquetipo_visual"]["arquetipo_indentidad"]
    ideal_yo = pd.isna(yo["arquetipo_visual"]["arquetipo_ideal"])
    
    if ideal_yo == identidad_match:
        frases.append(f"Además, este match encaja con el arquetipo que tú buscabas ({identidad_match}).")
    elif ideal_yo == "Me da igual":
        frases.append("Como indicaste que te daba igual el arquetipo, el algoritmo se centró en su gran afinidad contigo.")

    # --- CASO 3: Pareja Favorita ---
    pareja_yo = yo["arquetipo_visual"]["pareja_favorita"]
    pareja_match = match["arquetipo_visual"]["pareja_favorita"]
    
    if pareja_yo == pareja_match and pareja_yo != "Me da igual":
        frases.append(f"¡Increíble! Ambos comparten la misma pareja ideal de la ficción: {pareja_yo}.")
    elif pareja_yo == "Me da igual" or pareja_match == "Me da igual":
        # Caso donde uno o ambos son flexibles con la pareja ideal
        pass 

    # --- CASO 4: Ninguno coincide (Caso por defecto para que no quede vacío) ---
    if not frases:
        frases.append("Aunque tienen gustos visuales distintos, sus personalidades y descripciones conectaron profundamente.")

    return " ".join(frases)


In [ ]:
import yagmail

# Configura tu correo (Te recomiendo usar una App Password de Gmail)
# Google -> Seguridad -> Verificación en 2 pasos -> Contraseñas de aplicaciones
usuario_emisor = "uninortecis@gmail.com"
password_emisor = "lhcc ozbn texk ljbw" 

yag = yagmail.SMTP(usuario_emisor, password_emisor)

def enviar_correos_masivos(rankings_individuales, usuarios):
    for uid, matches in rankings_individuales.items():
        yo = usuarios[uid]
        email_destino = yo["metadata"]["correo"]
        nombre_yo = yo["metadata"]["nombre"]
        top_3 = matches[:3]
        
        # --- CUERPO DEL MENSAJE (HTML para que se vea lindo) ---
        cuerpo = f"<h2>¡Hola {nombre_yo}!</h2>"
        cuerpo += "<p>Gracias por participar en esta actividad. Tus tres matches con mayor puntaje son:</p><hr>"
        
        for i, m in enumerate(top_3, 1):
            el_match = usuarios[m["id_pareja"]]
            
            # Cálculos de ratios
            def calc_ratio(l1, l2):
                set_yo, set_m = set(l1), set(l2)
                return f"{len(set_yo.intersection(set_m))}/{len(set_yo)}"

            musica = calc_ratio(yo["intereses"]["musica"], el_match["intereses"]["musica"])
            hobbies = calc_ratio(yo["intereses"]["hobbies"], el_match["intereses"]["hobbies"])
            tiempo = calc_ratio(yo["intereses"]["tiempo_libre"], el_match["intereses"]["tiempo_libre"])
            
            # Explicaciones
            esc_txt = explicar_escalas(yo, el_match)
            vis_txt = explicar_arquetipos(yo, el_match)
            desc_match = el_match["descripcion"]["descripción_tipo"]

            # Bloque de Match
            cuerpo += f"""
            <div style='border: 1px solid #ddd; padding: 15px; margin-bottom: 10px; border-radius: 8px;'>
                <h3 style='color: #2e6c80;'>{i}. {m['match_con']} - Compatibilidad: {m['compatibilidad']}%</h3>
                <p><b>Contacto:</b> {m['contacto']}</p>
                <ul>
                    <li><b>Géneros musicales compartidos:</b> {musica}</li>
                    <li><b>Hobbies compartidos:</b> {hobbies}</li>
                    <li><b>Tiempo libre compartido:</b> {tiempo}</li>
                </ul>
                <p><i>{esc_txt}</i></p>
                <p><b>Visuales:</b> {vis_txt}</p>
                <p style='background: #f9f9f9; padding: 10px;'><b>Su persona ideal:</b> "{desc_match}" <br>
                <small>¿Crees que encajas con esta descripción?</small></p>
            </div>
            """
        
        cuerpo += "<p><br>¡Esperamos que disfrutes conectando!</p>"
        
        # Enviar
        try:
            yag.send(to=email_destino, subject="Tus Matches de la Actividad", contents=cuerpo)
            print(f"✅ Correo enviado a {nombre_yo} ({email_destino})")
        except Exception as e:
            print(f"❌ Error enviando a {nombre_yo}: {e}")


In [ ]:
enviar_correos_masivos(rankings_individuales, usuarios)

In [157]:
print(list(rankings_individuales.keys())[17])
yo = usuarios[list(rankings_individuales.keys())[17]]
nombre_yo = yo["metadata"]["nombre"]
print(nombre_yo)
matches = rankings_individuales[list(rankings_individuales.keys())[17]]
top_3 = matches[:3]
print(top_3)
#ver correo
print(yo["metadata"]["correo"])

id_18
Yader
[{'match_con': 'Gabriela Carbonell', 'id_pareja': 'id_127', 'compatibilidad': np.float32(37.3096), 'contacto': 'gaby_crbh', 'desglose': {'est': 21.946969696969695, 'vis': 10, 'txt': np.float32(5.3625917)}, 'correo_destino': 'yaderv@uninorte.edu.co'}, {'match_con': 'Alexandra Oliveros ', 'id_pareja': 'id_8', 'compatibilidad': np.float32(33.6916), 'contacto': '3241710296', 'desglose': {'est': 12.619047619047619, 'vis': 5, 'txt': np.float32(16.072561)}, 'correo_destino': 'yaderv@uninorte.edu.co'}, {'match_con': 'Daniela Roncallo ', 'id_pareja': 'id_110', 'compatibilidad': 31.9721, 'contacto': '3044185446', 'desglose': {'est': 21.972138680033417, 'vis': 10, 'txt': 0.0}, 'correo_destino': 'yaderv@uninorte.edu.co'}]
yaderv@uninorte.edu.co
